# Real-Time Sign Language Translator - Model Building & Training

This notebook builds and trains a CNN model for sign language recognition.

**Objectives:**
- Build CNN architecture (ResNet-based)
- Define loss function and optimizer
- Implement training loop with validation
- Track metrics and visualize training progress
- Save best model checkpoints
- Evaluate model performance

## 1. Import Libraries and Load Configuration

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from PIL import Image
import os
import json
import time
from tqdm import tqdm
import copy
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Load configuration
project_root = os.path.abspath('..')
config_file = os.path.join(project_root, 'config.json')

with open(config_file, 'r') as f:
    config = json.load(f)

# Load class mapping
class_mapping_file = os.path.join(project_root, 'class_mapping.json')
with open(class_mapping_file, 'r') as f:
    class_mapping = json.load(f)
    class_to_idx = class_mapping['class_to_idx']
    idx_to_class = {int(k): v for k, v in class_mapping['idx_to_class'].items()}

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config['device'] = device

print("✓ Configuration loaded successfully!")
print(f"Device: {device}")
print(f"Number of classes: {len(class_to_idx)}")
print(f"Project root: {project_root}")

✓ Configuration loaded successfully!
Device: cuda
Number of classes: 26
Project root: d:\Projects\RealTime-Sign-Language-Translator


## 2. Load Preprocessed Data

Load the DataLoaders created in the previous notebook.

In [ ]:
# Custom Dataset class (same as in previous notebook)
class SignLanguageDataset(Dataset):
    def __init__(self, data_path, transform=None, class_to_idx=None):
        self.data_path = data_path
        self.transform = transform
        self.class_to_idx = class_to_idx
        self.images = []
        self.labels = []
        
        for class_name in os.listdir(data_path):
            class_path = os.path.join(data_path, class_name)
            if not os.path.isdir(class_path):
                continue
            
            if class_to_idx and class_name in class_to_idx:
                class_idx = class_to_idx[class_name]
            else:
                class_idx = len(set(self.labels))
            
            for img_name in os.listdir(class_path):
                if img_name.endswith(('.jpg', '.png', '.jpeg')):
                    self.images.append(os.path.join(class_path, img_name))
                    self.labels.append(class_idx)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load data paths
processed_data_path = os.path.join(project_root, 'data', 'processed')
train_path = os.path.join(processed_data_path, 'train')
val_path = os.path.join(processed_data_path, 'val')
test_path = os.path.join(processed_data_path, 'test')

# Create datasets
print("Loading datasets...")
train_dataset = SignLanguageDataset(train_path, transform=train_transform, class_to_idx=class_to_idx)
val_dataset = SignLanguageDataset(val_path, transform=val_transform, class_to_idx=class_to_idx)
test_dataset = SignLanguageDataset(test_path, transform=val_transform, class_to_idx=class_to_idx)

# Create dataloaders
batch_size = int(config['batch_size'])
# Set num_workers=0 for Windows to avoid hanging issues
num_workers = 0  # int(config['num_workers'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                       num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                        num_workers=num_workers, pin_memory=True)

print(f"\n✓ Datasets loaded!")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Batch size: {batch_size}")

Loading datasets...

✓ Datasets loaded!
Training samples: 60900
Validation samples: 13050
Test samples: 13050
Batch size: 32


## 3. Build CNN Model Architecture

We'll use a pre-trained ResNet18 model and fine-tune it for sign language recognition.

In [3]:
class SignLanguageModel(nn.Module):
    def __init__(self, num_classes=26, pretrained=True):
        super(SignLanguageModel, self).__init__()
        
        # Load pre-trained ResNet18
        self.model = models.resnet18(pretrained=pretrained)
        
        # Modify the final fully connected layer
        num_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

# Create model
num_classes = len(class_to_idx)
model = SignLanguageModel(num_classes=num_classes, pretrained=True)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("="*60)
print("MODEL ARCHITECTURE")
print("="*60)
print(f"Base model: ResNet18 (pretrained on ImageNet)")
print(f"Number of classes: {num_classes}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("="*60)

print("\nModel structure:")
print(model)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\nasir/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [02:47<00:00, 279kB/s] 


MODEL ARCHITECTURE
Base model: ResNet18 (pretrained on ImageNet)
Number of classes: 26
Total parameters: 11,452,506
Trainable parameters: 11,452,506

Model structure:
SignLanguageModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock

## 4. Define Loss Function and Optimizer

Set up training components including loss, optimizer, and learning rate scheduler.

In [5]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
learning_rate = float(config['learning_rate'])
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                 factor=0.5, patience=3)

print("="*60)
print("TRAINING CONFIGURATION")
print("="*60)
print(f"Loss function: CrossEntropyLoss")
print(f"Optimizer: Adam")
print(f"Learning rate: {learning_rate}")
print(f"Weight decay: 1e-4")
print(f"LR scheduler: ReduceLROnPlateau")
print(f"  - Factor: 0.5")
print(f"  - Patience: 3")
print("="*60)

TRAINING CONFIGURATION
Loss function: CrossEntropyLoss
Optimizer: Adam
Learning rate: 0.001
Weight decay: 1e-4
LR scheduler: ReduceLROnPlateau
  - Factor: 0.5
  - Patience: 3


## 5. Training and Validation Functions

Define functions for training and validation loops.

In [6]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 
                         'acc': f'{100*correct/total:.2f}%'})
    
    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch(model, dataloader, criterion, device):
    """Validate for one epoch."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Update progress bar
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 
                             'acc': f'{100*correct/total:.2f}%'})
    
    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc

print("✓ Training and validation functions defined")

✓ Training and validation functions defined


## 6. Train the Model

Now let's train the model with early stopping and model checkpointing.

In [ ]:
# Training configuration
num_epochs = int(config['num_epochs'])
patience = int(config['early_stopping_patience'])
checkpoint_path = config['checkpoint_path']
model_path = config['model_path']

# Create checkpoint directory
os.makedirs(checkpoint_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'lr': []
}

# Early stopping variables
best_val_loss = float('inf')
best_model_wts = copy.deepcopy(model.state_dict())
epochs_no_improve = 0

print("="*60)
print("STARTING TRAINING")
print("="*60)
print(f"Number of epochs: {num_epochs}")
print(f"Early stopping patience: {patience}")
print(f"Checkpoint path: {checkpoint_path}")
print("="*60)

start_time = time.time()

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 60)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    # Print epoch summary
    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"  Learning Rate: {current_lr:.6f}")
    
    # Save best model
    if val_loss < best_val_loss:
        print(f"  ✓ Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}")
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
        
        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
            'history': history
        }
        torch.save(checkpoint, os.path.join(checkpoint_path, 'best_model.pth'))
        print(f"  ✓ Model checkpoint saved")
    else:
        epochs_no_improve += 1
        print(f"  No improvement for {epochs_no_improve} epoch(s)")
    
    # Early stopping
    if epochs_no_improve >= patience:
        print(f"\n⚠ Early stopping triggered after {epoch+1} epochs")
        break

# Load best model weights
model.load_state_dict(best_model_wts)

# Save final model
final_model_path = os.path.join(model_path, 'sign_language_model.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'class_to_idx': class_to_idx,
    'idx_to_class': idx_to_class,
    'model_config': {
        'num_classes': num_classes,
        'architecture': 'ResNet18'
    }
}, final_model_path)

training_time = time.time() - start_time

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"Total training time: {training_time/60:.2f} minutes")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Best validation accuracy: {history['val_acc'][history['val_loss'].index(best_val_loss)]:.2f}%")
print(f"Final model saved to: {final_model_path}")
print("="*60)

STARTING TRAINING
Number of epochs: 50
Early stopping patience: 10
Checkpoint path: d:\Projects\RealTime-Sign-Language-Translator\models\checkpoints

Epoch 1/50
------------------------------------------------------------


Training:   0%|          | 0/1904 [00:00<?, ?it/s]

## 7. Visualize Training History

Plot training and validation metrics over epochs.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs_range = range(1, len(history['train_loss']) + 1)

# Loss plot
axes[0].plot(epochs_range, history['train_loss'], 'b-', label='Training Loss', linewidth=2)
axes[0].plot(epochs_range, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(epochs_range, history['train_acc'], 'b-', label='Training Accuracy', linewidth=2)
axes[1].plot(epochs_range, history['val_acc'], 'r-', label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Learning rate plot
axes[2].plot(epochs_range, history['lr'], 'g-', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('Learning Rate', fontsize=12)
axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(config['output_path'], 'visualizations', 'training_history.png'), dpi=150)
plt.show()

print("✓ Training history visualized and saved")

## 8. Evaluate Model on Test Set

Evaluate the trained model on the test set and compute detailed metrics.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def evaluate_model(model, dataloader, device):
    """Evaluate model and return predictions and labels."""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Evaluating'):
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    return np.array(all_preds), np.array(all_labels)

# Evaluate on test set
print("Evaluating model on test set...")
test_preds, test_labels = evaluate_model(model, test_loader, device)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_preds) * 100

# Classification report
class_names = [idx_to_class[i] for i in range(len(idx_to_class))]
report = classification_report(test_labels, test_preds, target_names=class_names, digits=4)

print("\n" + "="*60)
print("TEST SET EVALUATION")
print("="*60)
print(f"Test Accuracy: {test_accuracy:.2f}%")
print("\nClassification Report:")
print(report)
print("="*60)

# Save metrics
metrics_path = os.path.join(config['output_path'], 'metrics')
os.makedirs(metrics_path, exist_ok=True)

with open(os.path.join(metrics_path, 'classification_report.txt'), 'w') as f:
    f.write(f"Test Accuracy: {test_accuracy:.2f}%\n\n")
    f.write("Classification Report:\n")
    f.write(report)

print(f"\n✓ Metrics saved to: {metrics_path}")

## 9. Confusion Matrix

Visualize the confusion matrix to understand model performance across classes.

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(test_labels, test_preds)

# Plot confusion matrix
plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.title('Confusion Matrix - Sign Language Recognition', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(config['output_path'], 'visualizations', 'confusion_matrix.png'), dpi=150)
plt.show()

# Calculate per-class accuracy
class_accuracy = cm.diagonal() / cm.sum(axis=1) * 100

# Display per-class accuracy
print("\nPer-Class Accuracy:")
print("="*60)
for i, cls in enumerate(class_names):
    print(f"{cls:3s}: {class_accuracy[i]:6.2f}% ({cm.diagonal()[i]:4d}/{cm.sum(axis=1)[i]:4d})")
print("="*60)

# Find best and worst performing classes
best_idx = np.argmax(class_accuracy)
worst_idx = np.argmin(class_accuracy)

print(f"\nBest performing class:  {class_names[best_idx]} ({class_accuracy[best_idx]:.2f}%)")
print(f"Worst performing class: {class_names[worst_idx]} ({class_accuracy[worst_idx]:.2f}%)")

print("\n✓ Confusion matrix visualized and saved")

## 10. Visualize Sample Predictions

Show model predictions on random test samples.

In [ ]:
def denormalize(tensor):
    """Denormalize image tensor for visualization."""
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean

def visualize_predictions(model, dataset, device, num_samples=12):
    """Visualize model predictions on random samples."""
    model.eval()
    
    # Get random indices
    indices = np.random.choice(len(dataset), num_samples, replace=False)
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.flatten()
    
    with torch.no_grad():
        for i, idx in enumerate(indices):
            image, true_label = dataset[idx]
            
            # Get prediction
            image_batch = image.unsqueeze(0).to(device)
            output = model(image_batch)
            _, pred_label = torch.max(output, 1)
            pred_label = pred_label.item()
            
            # Get confidence
            probs = torch.nn.functional.softmax(output, dim=1)
            confidence = probs[0, pred_label].item() * 100
            
            # Denormalize and convert to numpy
            img_denorm = denormalize(image)
            img_np = img_denorm.permute(1, 2, 0).numpy()
            img_np = np.clip(img_np, 0, 1)
            
            # Get class names
            true_class = idx_to_class[true_label]
            pred_class = idx_to_class[pred_label]
            
            # Plot
            axes[i].imshow(img_np)
            axes[i].axis('off')
            
            # Color code: green for correct, red for incorrect
            color = 'green' if true_label == pred_label else 'red'
            title = f'True: {true_class}\nPred: {pred_class} ({confidence:.1f}%)'
            axes[i].set_title(title, fontsize=11, color=color, fontweight='bold')
    
    plt.suptitle('Model Predictions on Test Set', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(config['output_path'], 'visualizations', 'sample_predictions.png'), dpi=150)
    plt.show()

# Visualize predictions
visualize_predictions(model, test_dataset, device, num_samples=12)

print("✓ Sample predictions visualized and saved")

## 11. Summary

Training and evaluation complete! Model is ready for real-time inference.

In [ ]:
print("="*70)
print("MODEL TRAINING AND EVALUATION SUMMARY")
print("="*70)

print("\n📊 DATASET:")
print(f"  Training samples:   {len(train_dataset):,}")
print(f"  Validation samples: {len(val_dataset):,}")
print(f"  Test samples:       {len(test_dataset):,}")
print(f"  Number of classes:  {num_classes}")

print("\n🏗️  MODEL ARCHITECTURE:")
print(f"  Base model:          ResNet18 (pretrained)")
print(f"  Total parameters:    {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

print("\n⚙️  TRAINING CONFIGURATION:")
print(f"  Optimizer:           Adam")
print(f"  Learning rate:       {learning_rate}")
print(f"  Batch size:          {batch_size}")
print(f"  Epochs trained:      {len(history['train_loss'])}")
print(f"  Training time:       {training_time/60:.2f} minutes")

print("\n📈 PERFORMANCE METRICS:")
print(f"  Best validation loss:     {best_val_loss:.4f}")
print(f"  Best validation accuracy: {max(history['val_acc']):.2f}%")
print(f"  Test accuracy:            {test_accuracy:.2f}%")
print(f"  Best performing class:    {class_names[best_idx]} ({class_accuracy[best_idx]:.2f}%)")
print(f"  Worst performing class:   {class_names[worst_idx]} ({class_accuracy[worst_idx]:.2f}%)")

print("\n💾 SAVED FILES:")
print(f"  Model checkpoint:     {os.path.join(checkpoint_path, 'best_model.pth')}")
print(f"  Final model:          {final_model_path}")
print(f"  Training history:     {os.path.join(config['output_path'], 'visualizations', 'training_history.png')}")
print(f"  Confusion matrix:     {os.path.join(config['output_path'], 'visualizations', 'confusion_matrix.png')}")
print(f"  Classification report: {os.path.join(metrics_path, 'classification_report.txt')}")

print("\n" + "="*70)
print("NEXT STEPS:")
print("="*70)
print("1. Test model on real-time webcam input")
print("2. Implement real-time sign language detection")
print("3. Create user interface for the application")
print("4. Deploy the model for production use")
print("="*70)

# Save training history
history_df = pd.DataFrame(history)
history_df.to_csv(os.path.join(metrics_path, 'training_history.csv'), index=False)
print(f"\n✓ Training history saved to: {os.path.join(metrics_path, 'training_history.csv')}")